In [ ]:
import cv2
import numpy as np
import scipy.ndimage
import scipy.stats
import time
import multiprocessing
import ImportNotebook
import GenerateDaugmanTemplateSet
import DaugmanIrisCore
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import copy as cp


mypool =  multiprocessing.Pool()
#？这几个变量的含义
imageWorkingWidth = 150.0
irisRangeLB = 0.16
irisRangeUB = 0.24
irisTemplateSet = []
irisWeights = []
irisRange = []
irisRadiiLog = []
efficient=np.array([1,0.2,0.2,0.9])

def regulateIrisRange():
    from scipy.stats import norm
    #得到随机序列的期望和标准差，更改原理是啥？
    (mu, sigma) = norm.fit(irisRadiiLog)
    global irisRangeLB
    global irisRangeUB
    irisRangeLB = mu - 2*sigma
    irisRangeUB = mu + 2*sigma
    generateTemplateSet()
    #generateTemplateSet2()#新模板不用再次生成，用旧的就好

def generateTemplateSet():
    global irisTemplateSet
    global irisWeights
    global irisRange
    irisRange = np.arange(round(irisRangeLB*imageWorkingWidth),round(irisRangeUB*imageWorkingWidth)+1,1)
    irisRingWidth = 1
    GenerateDaugmanTemplateSet.GenerateDaugmanTemplateSet.pool = mypool
    irisTemplateSet,irisWeights = GenerateDaugmanTemplateSet.GenerateDaugmanTemplateSet(irisRingWidth,irisRange)
    #print 'irisTemplateSet'
    #print len(irisTemplateSet[0])
    #print len(irisTemplateSet[1])

def generateTemplateSet2():
    global irisTemplateSet2
    global irisWeights2
    global irisRange2
    irisRange2 = np.arange(round(irisRangeLB*imageWorkingWidth),round(irisRangeUB*imageWorkingWidth)+1,1)
    irisRingWidth = 1
    irisTemplateSet2,irisWeights2,index2 = GenerateDaugmanTemplateSet.GenerateDaugmanTemplateSet2(irisRingWidth,irisRange)
    #print 'irisTemplateSet2'
    #print len(irisTemplateSet2)
    
def EqualizeRGBImage(img):
    #转化成YUV通道
    ycrcb = cv2.cvtColor(img,cv2.COLOR_RGB2YCR_CB)
    #灰度图直方图均衡，第0层是亮度
    ycrcb[:,:,0] = cv2.equalizeHist(ycrcb[:,:,0])
    #再转化回来
    return cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2RGB)


def DaugmanIrisLocalization(img,count,num): 
    countStr='%d'%count
    numStr='%d'%num
    #cv2.imwrite('/Users/ly/Code/picBefore/eye'+countStr+'_'+numStr+'.jpg',img)
    #做了亮度的处理，为了更好的找到边界
    imgeq = EqualizeRGBImage(img)
    #cv2.imshow('test1',img)
    imglarge = cv2.resize(imgeq,(np.int(imageWorkingWidth),np.int(np.ceil(imageWorkingWidth/imgeq.shape[1]*imgeq.shape[0]))),interpolation=cv2.INTER_LINEAR)
    #cv2.imshow('test2',imglarge)
    #保护边缘的平滑滤波器bilateralFilter
    imgfiltered = cv2.bilateralFilter(imglarge,5,150,50)
    #cv2.imshow('test3',imgfiltered)
    imggrey = cv2.cvtColor(imgfiltered,cv2.COLOR_RGB2GRAY)
    #cv2.imshow('test4',imggrey)
    cv2.imwrite('/Users/ly/Code/picAfter/eye'+countStr+'_'+numStr+'.jpg',imggrey)
    
    covstackDiffSet = []
    covstackdiffmaxmaps = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
    covstackdiffmaxradii = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
    #print '-' *40
    #print imggrey.shape      #(104,150)或者(105,150)或者(106,150)
    #print list(imggrey.shape) #[104,150]
    #print [len(irisTemplateSet)]    #[2]
    #print list(imggrey.shape) + [len(irisTemplateSet)]  #[104,150,2]
    #print covstackdiffmaxmaps.shape  #(104,150,2)
    #print covstackdiffmaxmaps
    #print '-' *40
        
    results = [mypool.apply_async(DaugmanIrisCore.DaugmanIrisCore,(irisTemplates,imggrey)) for irisTemplates in irisTemplateSet]
    parallelResults = [r.get() for r in results]
        
    for s in xrange(len(parallelResults)):
        covstackdiffmaxmaps[:,:,s] = parallelResults[s][0]
        covstackdiffmaxradii[:,:,s] = parallelResults[s][1]
        covstackDiffSet.append(parallelResults[s][2])
    
    covstackdiffmaxsum = 0
    for s in xrange(len(irisTemplateSet)):
        covstackdiffmaxsum += covstackdiffmaxmaps[:,:,s]*irisWeights[s]
    covstackdiffmaxsum **= 3
    covstackdiffmaxsumfiltered = scipy.ndimage.gaussian_filter(covstackdiffmaxsum,6,0,None,mode='nearest')
    DaugmanIrisCenter = np.unravel_index(np.argmax(covstackdiffmaxsumfiltered),covstackdiffmaxsumfiltered.shape)
    DaugmanIrisRadius = irisRange[np.round(scipy.stats.nanmedian(covstackdiffmaxradii[DaugmanIrisCenter[0],DaugmanIrisCenter[1],:]))]
    global irisRadiiLog
    irisRadiiLog.append(DaugmanIrisRadius/imageWorkingWidth)
    DaugmanIrisCenter = DaugmanIrisCenter[::-1]
    #分别计算出眼珠中心的坐标和黑眼仁的半径
    eyecenter = np.float64(DaugmanIrisCenter) * (img.shape[1]/imageWorkingWidth)
    eyeiris =   np.float64(DaugmanIrisRadius) * (img.shape[1]/imageWorkingWidth)
    print type(DaugmanIrisCenter)
    print DaugmanIrisCenter
    print type(DaugmanIrisCenter[0])
    print type(DaugmanIrisRadius)
    print DaugmanIrisRadius
    #cv2.circle(imggrey,tuple(np.int32(np.squeeze(DaugmanIrisCenter))),1,(0,0,255))
    #cv2.circle(imggrey,tuple(np.int32(np.squeeze(DaugmanIrisCenter))),np.int32(DaugmanIrisRadius),(0,0,255))
    #cv2.imshow('hello',imggrey)
    return eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered

def fillHoleInBinary(Img,binaryImg):
    contours,hierarchy=cv2.findContours(binaryImg,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
    if len(contours)!=0:
        for i in xrange(0,len(contours)):
            if len(contours[i])<100:
                cv2.drawContours(Img,contours,i,0,-1)
    else:
        print 'no edge is detected!'
    return Img

def UseKmeans(noHoleImg,grayImg):
    l1=np.reshape(noHoleImg[:,:].T,(grayImg.shape[0]*grayImg.shape[1],1))
    x=np.reshape(np.tile(np.arange(grayImg.shape[0]),grayImg.shape[1]),(l1.shape[0],1))
    y=np.reshape(np.arange(grayImg.shape[1]).repeat(grayImg.shape[0]),(l1.shape[0],1))
    data=np.hstack((l1*efficient[0],x*efficient[1],y*efficient[1]))
    #the first time to operate kmeans
    kmeans=KMeans(n_clusters=2,random_state=0).fit(data)
    labels=kmeans.predict(data)
    #find the XXX
    if np.mean(data[np.where(labels==1),0])>np.mean(data[np.where(labels==0),0]):
        ti=0
    else:
        ti=1
    labels[np.where(labels!=ti)]=2
    newdata=data[np.where(labels==ti)]
    newdataIndex=np.where(labels==ti)[0]
    #the second time to operate kmeans
    kmeans=KMeans(n_clusters=2,random_state=0).fit(newdata)
    newlabels=kmeans.predict(newdata)
    if np.mean(newdata[np.where(newlabels==1),0])>np.mean(newdata[np.where(newlabels==0),0]):
        labels[newdataIndex[np.where(newlabels==1)[0]]]=0
        labels[newdataIndex[np.where(newlabels==0)[0]]]=1
    else:
        labels[newdataIndex[np.where(newlabels==0)[0]]]=0
        labels[newdataIndex[np.where(newlabels==1)[0]]]=1
    labels=np.uint8(255-np.reshape(labels,(grayImg.shape[1],grayImg.shape[0])).T*127)
    cv2.imshow('kmeans',labels)
    return labels

#center:<type 'tuple'><type 'numpy.int64'>    radius:<type 'numpy.float64'>
def caculateShelterRate(center,radius,img):
    topShelter=[]
    bottomShelter=[]
    radius=int(round(radius))
    topedge=center[1]-radius
    bottomedge=center[1]+radius
    print '_'*40
    print type(topedge)
    print type(center[1])
    print type(radius)
    #print range(topedge,center[1])[::-1]
    for i in range(topedge,center[1])[::-1]:#[49-17]
        if img[i,center[0]]!=128:
            topPos=i
            break
    #print range(center[1]+1,bottomedge+1)
    for i in range(center[1]+1,bottomedge+1):#[51-83]
        if img[i,center[0]]!=128:
            bottomPos=i
            break
    topShelter.append(center[1]-topPos-1)
    topShelter.append(center[1]-topPos)
    bottomShelter.append(bottomPos-center[1]-1)
    bottomShelter.append(bottomPos-center[1])
    
    return topShelter,bottomShelter
    
def DaugmanIrisLocalization1(img): 
    imgeq = EqualizeRGBImage(img)
    imglarge = cv2.resize(imgeq,(np.int(imageWorkingWidth),np.int(np.ceil(imageWorkingWidth/imgeq.shape[1]*imgeq.shape[0]))),interpolation=cv2.INTER_LINEAR)
    imgfiltered = cv2.bilateralFilter(imglarge,5,150,50)
    imggrey = cv2.cvtColor(imgfiltered,cv2.COLOR_RGB2GRAY)
    
    covstackDiffSet = []
    covstackdiffmaxmaps = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
    covstackdiffmaxradii = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
        
    results = [mypool.apply_async(DaugmanIrisCore.DaugmanIrisCore,(irisTemplates,imggrey)) for irisTemplates in irisTemplateSet]
    parallelResults = [r.get() for r in results]
        
    for s in xrange(len(parallelResults)):
        covstackdiffmaxmaps[:,:,s] = parallelResults[s][0]
        covstackdiffmaxradii[:,:,s] = parallelResults[s][1]
        covstackDiffSet.append(parallelResults[s][2])
    
    covstackdiffmaxsum = 0
    for s in xrange(len(irisTemplateSet)):
        covstackdiffmaxsum += covstackdiffmaxmaps[:,:,s]*irisWeights[s]
    covstackdiffmaxsum **= 3
    covstackdiffmaxsumfiltered = scipy.ndimage.gaussian_filter(covstackdiffmaxsum,6,0,None,mode='nearest')
    DaugmanIrisCenter = np.unravel_index(np.argmax(covstackdiffmaxsumfiltered),covstackdiffmaxsumfiltered.shape)
    DaugmanIrisRadius = irisRange[np.round(scipy.stats.nanmedian(covstackdiffmaxradii[DaugmanIrisCenter[0],DaugmanIrisCenter[1],:]))]
    global irisRadiiLog
    irisRadiiLog.append(DaugmanIrisRadius/imageWorkingWidth)
    DaugmanIrisCenter = DaugmanIrisCenter[::-1]
    #分别计算出眼珠中心的坐标和黑眼仁的半径
    eyecenter = np.float64(DaugmanIrisCenter) * (img.shape[1]/imageWorkingWidth)
    eyeiris =   np.float64(DaugmanIrisRadius) * (img.shape[1]/imageWorkingWidth)
    
    ret,imgBinary=cv2.threshold(imggrey,70,255,cv2.THRESH_BINARY)
    imgNoHole=fillHoleInBinary(imggrey,imgBinary)
    cv2.imshow('imgNoHole',imgNoHole)
    imgKmeans=UseKmeans(imgNoHole,imggrey)
    #计算出上下遮挡比例
    topShelterRate,bottomShelterRate=caculateShelterRate(DaugmanIrisCenter,DaugmanIrisRadius,imgKmeans)
    
    return eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered,topShelterRate,bottomShelterRate

def DaugmanIrisLocalization2(img,info): 
    imgeq = EqualizeRGBImage(img)
    imglarge = cv2.resize(imgeq,(np.int(imageWorkingWidth),np.int(np.ceil(imageWorkingWidth/imgeq.shape[1]*imgeq.shape[0]))),interpolation=cv2.INTER_LINEAR)
    imgfiltered = cv2.bilateralFilter(imglarge,5,150,50)
    imggrey = cv2.cvtColor(imgfiltered,cv2.COLOR_RGB2GRAY)
    
    covstackDiffSet = []
    covstackdiffmaxmaps = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
    covstackdiffmaxradii = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
        
    results = [mypool.apply_async(DaugmanIrisCore.DaugmanIrisCore,(irisTemplates,imggrey)) for irisTemplates in irisTemplateSet]
    parallelResults = [r.get() for r in results]
        
    for s in xrange(len(parallelResults)):
        covstackdiffmaxmaps[:,:,s] = parallelResults[s][0]
        covstackdiffmaxradii[:,:,s] = parallelResults[s][1]
        covstackDiffSet.append(parallelResults[s][2])
    
    covstackdiffmaxsum = 0
    for s in xrange(len(irisTemplateSet)):
        covstackdiffmaxsum += covstackdiffmaxmaps[:,:,s]*irisWeights[s]
    covstackdiffmaxsum **= 3
    covstackdiffmaxsumfiltered = scipy.ndimage.gaussian_filter(covstackdiffmaxsum,6,0,None,mode='nearest')
    DaugmanIrisCenter = np.unravel_index(np.argmax(covstackdiffmaxsumfiltered),covstackdiffmaxsumfiltered.shape)
    DaugmanIrisRadius = irisRange[np.round(scipy.stats.nanmedian(covstackdiffmaxradii[DaugmanIrisCenter[0],DaugmanIrisCenter[1],:]
    ))]
    global irisRadiiLog
    irisRadiiLog.append(DaugmanIrisRadius/imageWorkingWidth)
    DaugmanIrisCenter = DaugmanIrisCenter[::-1]
    #分别计算出眼珠中心的坐标和黑眼仁的半径
    eyecenter = np.float64(DaugmanIrisCenter) * (img.shape[1]/imageWorkingWidth)
    eyeiris =   np.float64(DaugmanIrisRadius) * (img.shape[1]/imageWorkingWidth)
    topShelterRate=[]
    bottomShelterRate=[]
    
    return eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered,topShelterRate,bottomShelterRate

def main():
    img = mpimg.imread('/Users/Jiangzp/test.jpg')
    initialize()
    t = time.time()
    for x in range(0,20):
        eyecenter,eyeiris = DaugmanIrisLocalization(img)
    print time.time() -t
    print eyecenter,eyeiris
    
    mypool.close()
    mypool.join()

    plt.imshow(img)
    plt.show()

def localizeIrisAndShow(img):
    initialize()
    eyecenter,eyeiris = DaugmanIrisLocalization(img)
    cv2.circle(img,tuple(eyecenter),int(eyeiris),(0,0,255))
    plt.imshow(img)
    plt.show()
